# Pytorch Tensor

In [4]:
import torch
import numpy as np

# Tensor的属性

<img src="../images/tensor_attributes.png" width=550px>

## dtype

torch在创建Tensor时，`dtype`的指定只支持使用`torch.[DataType]`这样的方式去指定，而不能像numpy一样，可以直接使用字符串。

In [2]:
torch.tensor([1, 2, 3], dtype=torch.int64)

tensor([1, 2, 3])

In [3]:
np.array([1, 2, 3], dtype="int64")

array([1, 2, 3])

In [4]:
a = torch.CharTensor([1, 2, 3])
a.dtype

torch.int8

其中`torch.dtype`表示Tensor的数据类型，常见的有下面9种不同的数据类型，包括了

- `torch.float32`或`torch.float`，对应的Tensor类型为`torch.[cuda].FloatTensor`
- `torch.float64`或`torch.double`，对应的Tensor类型为`torch.[cuda].DoubleTensor`
- `torch.float16`或`torch.half`，对应的Tensor类型为`torch.[cuda].HalfTensor`
- `torch.uint8`，对应的Tensor类型为`torch.[cuda].ByteTensor`
- `torch.int8`，对应的Tensor类型为`torch.[cuda].CharTensor`
- `torch.int16`或`torch.short`，对应的Tensor类型为`torch.[cuda].ShortTensor`
- `torch.int32`或`torch.int`，对应的Tensor类型为`torch.[cuda].IntTensor`
- `torch.int64`或`torch.long`，对应的Tensor类型为`torch.[cuda].LongTensor`
- `torch.bool`，对应的Tensor类型为`torch.[cuda].BoolTensor`

还有几种数据类型，用的比较少：

- `torch.bfloat16`
- `torch.complex32`: complex32 目前只是实验性的支持，后续可能会取消支持
- `torch.complex64`
- `torch.complex128`或`torch.cdouble`

In [5]:
import torch

# 创建一个 Complex64 张量
real_part = torch.randn(3, 3)  # 实部
imag_part = torch.randn(3, 3)  # 虚部
complex_tensor = torch.complex(real_part, imag_part)

print(complex_tensor)
print(complex_tensor.dtype)  # 输出: torch.complex64
complex_tensor = complex_tensor.to(dtype=torch.cdouble)
print(complex_tensor.dtype)  # 输出: torch.complex128

tensor([[ 0.7482-0.1826j, -0.7404+0.7532j, -0.2283-0.3117j],
        [ 0.4588+1.8799j, -1.1133+0.1339j, -0.1241+0.0917j],
        [ 0.9690+0.4072j,  1.0455-0.2295j,  1.0468-1.6390j]])
torch.complex64
torch.complex128


[`bfloat16`](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format)是一种和IEEE half-precision 16-bit float规定不一致的16Bit浮点数格式，它是直接对32位的IEEE 754规定的单精度float32的格式进行截取形成的，它是为机器学习系统特别定制的，它的组成是：
- 1位符号位
- 8个指数位
- 7个小数位

神经网络对指数的大小比对尾数的大小更敏感。为了确保下溢、上溢和`NaN`的行为完全相同，`bfloat16`的指数大小与`float32`相同。`bfloat16`处理非规格化数的规则与`float32`不同，它会将它们截断为零。与通常需要特殊处理，如损失缩放的`float16`不同，`bfloat16`是训练和运行深度神经网络时`float32`的即插即用替换。

简而言之，`bfloat16`表示的数值范围更大，但是精度不如`float16`

<center style="width: 100%"><img src="../images/floating_point_formats.png" width="800px"></center>


在不同的机器上，因为CPU架构等不同，Tensor的很多构建函数，对上面的部分`dtype`有可能是不支持的，比如`arange`函数就不支持在`cpu`上创建一个`float16`的Tensor。（新版本已经支持了）

In [6]:
torch.arange(1, 10, dtype=torch.float16, device=torch.device("cpu"))

tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=torch.float16)

由于CUDA对半精度支持的比较好，所以在'cuda'上创建，反而没有什么问题

In [7]:
torch.arange(1, 10, dtype=torch.float16, device=torch.device("cuda"))

tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.], device='cuda:0',
       dtype=torch.float16)

另外，在 Torch 2.x 的版本中也开始实验性的支持`FP8`格式，目前支持两种类型的`FP8`格式，一种是`E3M4`一种是`E5M2`，其中`E5M2`和 IEEE 的 `fp16`的指数部分长度一致，这使得 E5M2 和 IEEE FP16 格式之间可以直接转换。`E5M2`它表达的数值范围更大，可以处理一些特殊值。

一般来说，我们会在模型推理以及训练过程的前向阶段为了保持较高的精度会使用`E3M4`的格式，而在训练的反向阶段，使用`E5M2`来有较好的数值动态范围,避免梯度消失与爆炸。

Paper: [FP8 Formats for Deep Learning](https://arxiv.org/abs/2209.05433)

<img src="../images/torch_fp8.png" width="600px">

上图展示了使用不同数值类型来表示“0.3952” 时，实际能够近似到的值。

In [8]:
import torch
from tabulate import tabulate

f32_type = torch.float32
bf16_type = torch.bfloat16
e4m3_type = torch.float8_e4m3fn
e5m2_type = torch.float8_e5m2

# collect finfo for each type
table = []
for dtype in [f32_type, bf16_type, e4m3_type, e5m2_type]:
    numbits = 32 if dtype == f32_type else 16 if dtype == bf16_type else 8
    info = torch.finfo(dtype)
    table.append(
        [info.dtype, numbits, info.max, info.min, info.smallest_normal, info.eps]
    )

headers = ["data type", "bits", "max", "min", "smallest normal", "eps"]
print(tabulate(table, headers=headers))

data type        bits              max               min    smallest normal          eps
-------------  ------  ---------------  ----------------  -----------------  -----------
float32            32      3.40282e+38      -3.40282e+38        1.17549e-38  1.19209e-07
bfloat16           16      3.38953e+38      -3.38953e+38        1.17549e-38  0.0078125
float8_e4m3fn       8    448              -448                  0.015625     0.125
float8_e5m2         8  57344            -57344                  6.10352e-05  0.25


## device

`torch.device`表示的是Tensor的数据存储的设备，其中分为`cpu`和`cuda`

In [9]:
torch.device("cpu")

device(type='cpu')

In [10]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [11]:
torch.device(type="cuda", index=0)

device(type='cuda', index=0)

In [12]:
torch.tensor([1, 2, 3, 4, 5, 6], device=torch.device("cuda:0"))

tensor([1, 2, 3, 4, 5, 6], device='cuda:0')

## layout

layout表示Tensor内部数据存储的内部布局，目前还是一个不成熟(beta)的特性，目前支持

- torch.strided
- torch.sparse_coo

现在主要用的就是面向dense Tensor的`torch.strided`，Tensor的Strides是一个list，它代表每个dimension上两邻两个idx之间的跨度(元素个数)。

In [13]:
torch.arange(60).reshape(3, 4, 5).stride()

(20, 5, 1)

我们可以理解为 Tensor 底层的存储的是一个一维的数组，我们对于 `Tensor`的索引，全部是是通过一个下标对应的 stride 来计算出最终在一维数组上的偏移量。 这样实现的好处时，对于 `Tensor`的很多操作，它并不需要实际对 `Tensor`的内存数据进行变动。

In [14]:
t = torch.arange(12).view(3, 4)
# t_transposed 和 t 共享底层的数据
t_transposed = t.transpose(0, 1)
print(t.stride())
print(t_transposed.stride())

(4, 1)
(1, 4)


和numpy不同的是，torch中的stride以元素的个数来表示跨度，而numpy则是用字节数量来表示跨度

In [15]:
np.arange(60).reshape(3, 4, 5).strides

(160, 40, 8)

## Tensor属性转换

我们可以使用`to`方法来指定新的属性后，生成新的Tensor

In [16]:
device_cuda = torch.device("cuda")
data = torch.tensor([1])
print(data.dtype, data.device)
data = data.to(dtype=torch.float32, device=device_cuda)
print(data.dtype, data.device)

torch.int64 cpu
torch.float32 cuda:0


也可以通过 Tensor 的`dtype`方法来直接将返回新数据类型的 Tensor

In [17]:
data.int()
data.float()
data.bool()

tensor([True], device='cuda:0')

## Tensor的形状

Tensor除了具有3个标准的属性外，一旦我们创建了一个Tensor，那么它就会具有一些形状相关的属性。

- t.shape: 返回的是一个torch.Size(tuple)类型的结果，表示每一维的维度值
- t.size(): 和t.shape一致
- t.size(i): 返回第 i 个维度的值
- t.ndim：返回Tensor有多少维
- t.numel()：它是一个方法，返回Tensor内有多少个元素
- len(t)：返回的是Tensor在第0维上的维度值

In [18]:
t = torch.empty(2, 3, 4)
print(f"shape of t is {t.shape}")
print(f"size of t is {t.size()}")
print(f"size(1) of t is {t.size(1)}")
print(f"strides of t is {t.stride()}")
print(f"strides of axes{1} of t is {t.stride(1)}")
print(f"ndim of t is {t.ndim}")
print(f"numel of t is {t.numel()}")
print(f"len of t is {len(t)}")

shape of t is torch.Size([2, 3, 4])
size of t is torch.Size([2, 3, 4])
size(1) of t is 3
strides of t is (12, 4, 1)
strides of axes1 of t is 4
ndim of t is 3
numel of t is 24
len of t is 2


# Tensor的创建

在Pytorch中我们可以有多种方法来创建Tensor，常用的包括下面几种：

- 从已有的scalar、list、tuple、numpy.array来创建
- 用`arange`、`linspace`、`logspace`等创建一维数列Tensor
- 用`ones`、`zeros`、`eye`、`full`、`empty`等来创建特别填充值的多维Tensor
- 用随机数来创建指定形状的Tensor

![](../images/tensor_creation.png)

## 从现有数据来创建一个Tensor

我们可以使用`torch.tensor()`函数来从已有的一个array_like的data来创建一个Tensor

In [19]:
# 从list创建
torch.tensor([1, 2, 3, 4, 5])

tensor([1, 2, 3, 4, 5])

In [20]:
# 从tuple创建
torch.tensor((1, 2, 3))

tensor([1, 2, 3])

In [21]:
# 从numpy.array创建，同时指定dtype和device
torch.tensor(np.array([1, 2, 3, 4, 5]), dtype=torch.float32, device="cuda:0")

tensor([1., 2., 3., 4., 5.], device='cuda:0')

需要注意的是，无论是从python的内置序列创建，还是从numpy.array来创建，创建出来的Tensor都是复制了原数据的内容。

如果我们希望，创建的Tensor不额外分配存储空间，而是和之前的numpy.array共享存储，那么可以使用`as_tensor`方法

In [22]:
arr = np.array([1, 2, 3, 4, 5])
t = torch.as_tensor(arr)
# 对于Tensor的数据改动，也会影响在ndarray上
t[0] = 6
print(arr)

[6 2 3 4 5]


不过使用`as_tensor`后，能共享底层存储的，前提是，as_type方法中指定的`dtype`和`device`和原ndarry是一致的。由于numpy不支持cuda，所以这样只能创建cpu上的tensor

In [23]:
arr = np.array([1, 2, 3, 4, 5])
t = torch.as_tensor(arr, dtype=torch.float32)  # 这种情况下，并不会共享底层存储
t[0] = 6
print(arr)

[1 2 3 4 5]


In [24]:
il = [1, 2, 3, 4, 5]
print(f"ndarray的默认整数类型为:{np.array(il).dtype}")
print(f"tensor的默认整数类型为: {torch.tensor(il).dtype}")

fl = [1.0, 2.0, 3.0, 4.0, 5.0]
print(f"ndarray的默认整数类型为:{np.array(fl).dtype}")
print(f"tensor的默认整数类型为: {torch.tensor(fl).dtype}")

ndarray的默认整数类型为:int64
tensor的默认整数类型为: torch.int64
ndarray的默认整数类型为:float64
tensor的默认整数类型为: torch.float32


In [25]:
# 从另外一个tensor来创建tensor，无论 b 是否指新新的 dtype 和 device，b 都不和 a 共享数据
a = torch.tensor([1, 2, 3])
b = torch.tensor(a, dtype=torch.float, device="cuda:1")
b

/tmp/ipykernel_219404/2406181639.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, dtype=torch.float, device="cuda:1")


tensor([1., 2., 3.], device='cuda:1')

In [26]:
b = a.clone()
b = b.to(device="cuda:1")
print(b)

tensor([1, 2, 3], device='cuda:1')


## `torch.tensor()`和`torch.Tensor()`的区别

`torch.Tensor`实际上是`torch.FloatTensor`，用它来创建新的Tensor时，实际调用的是构造函数，它会默认以`torch.float32`来作为`dtype`。而`torch.tensor`会根据`data`的类型自动推断。

In [27]:
l = [1, 2, 3, 4, 5]
print(torch.Tensor(l).dtype)
print(torch.tensor(l).dtype)

torch.float32
torch.int64


## 创建特别填充值的Tensor

### torch.arange

torch.arange(start=0, end, step=1)用于创建一个区间范围的Tensor

In [28]:
print(torch.arange(5))
print(torch.arange(1, 5))
print(torch.arange(1, 20, 3))

tensor([0, 1, 2, 3, 4])
tensor([1, 2, 3, 4])
tensor([ 1,  4,  7, 10, 13, 16, 19])


In [29]:
# 如果start、end以及step中有浮点数，则创建出来的是FloatTensor
torch.arange(1, 3.5, 0.5)

tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000])

注意上面是没有包括3.5那个点的

### torch.linspace

`torch.linspace`与`torch.arange`有点类似，都指定一个起点，一个终点，和一个步长。但`linspace`里步长最终指定了生成的一维Tensor中元素的个数

```python
linspace(start(float),end(float),steps(int))
```
另外需要注意的是`torch.linspace`生成的一定是一个浮点数的Tensor，而且和`torch.arange`不同的是：`linspace`生成的Tensor是包括末点值的（inclusive）

In [30]:
torch.linspace(3, 10, 5)

tensor([ 3.0000,  4.7500,  6.5000,  8.2500, 10.0000])

### torch.logspace

`torch.logspace`和`torch.linspace`行为类似，区别在于`logspace`生成的序列的范围的起始与终点是一个以`base`为底，`start`和`end`为指数的数字。

```python
logspace(start, end, stpes, base=10.0) -> Tensor
```

### torch.ones、torch.zeros、torch.emtpy

它们三个都是用于创建一个指定`size`的Tensor，分别以1、0和未初始化的值来填充

它们三个返回的都是`FloatTensor`

In [31]:
print(torch.ones((2, 2)))
print(torch.zeros((3, 4)))
print(torch.empty((3, 3)))

tensor([[1., 1.],
        [1., 1.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[6.0555e-32, 0.0000e+00, 6.0717e-32],
        [0.0000e+00, 4.0357e-43, 0.0000e+00],
        [1.5835e-43, 0.0000e+00, 1.0483e-13]])


`torch.ones/zeros/empty`支持`torch.ones(d1,d2,...)`这种调用方法，而`numpy`则不支持。

### torch.eye

`torch.eye`返回的是一个2d的对角线为1，其他值都为0的Float矩阵Tensor

In [32]:
torch.eye(4)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

### torch.full

`torch.full`返回的是一个指定`size`和填充值的Tensor，Tensor的dtype是由填充值的类型来推导的。

```python
'''
Args:
  size(int...): a list ,tuple or torch.Size
  fill_vale(Scalar)
'''
full(size, fill_value)
```

In [33]:
torch.full((2, 3), 1.0)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

### torch.diag

如果输入的是一个 1d 的 Tensor，则返回的是一个 2d 的对角矩阵，其对角线上的元素为传入的 Tensor

In [34]:
torch.diag(torch.tensor([1, 2, 3]))

tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])

In [35]:
torch.diag(torch.tensor([1, 2, 3]), diagonal=-1)

tensor([[0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 2, 0, 0],
        [0, 0, 3, 0]])

传入 2d 的 Tensor，则返回 Tensor 的对角线上的元素，返回的是一个 1d 的 Tensor

In [36]:
torch.diag(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

tensor([1, 5, 9])

In [37]:
torch.diag(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), diagonal=1)

tensor([2, 6])

## 使用随机数来创建Tensor

### torch.normal

`torch.normal`返回一个正态分布产生在的随机数填充的Tensor，它一共有4种参数传递方式

第一种是:

```python
'''
Args:
    mean (Tensor): the tensor of per-element means
    std (Tensor): the tensor of per-element standard deviations
'''
norm(mean, std)
```
生成的Tensor的size和mean和std的size是一致的，其中每个元素都是通过对应位置的mean和std形成的正态分布来随机产生的。

In [38]:
mean = torch.randn(3, 4)
std = torch.rand((3, 4))
torch.normal(mean, std)

tensor([[ 1.4107, -1.0148, -0.7799,  1.8444],
        [-0.3895,  1.2914, -1.8545, -0.2155],
        [-0.9079, -0.1627,  0.4883,  0.2391]])

第二种是：

```python
'''
Args:
    mean (float, optional): the mean for all distributions
    std (Tensor): the tensor of per-element standard deviations
'''
normal(mean=0.0, std, *, out=None) -> Tensor
```
这种参数传递用法，与上面的区别就是mean变成一个Scalar，那么说明每个元素来共享一个mean值。

在这种情况下，生成的Tensor的shape就行std保持一致的了。

In [39]:
torch.normal(1.0, std)

tensor([[0.6667, 1.3038, 0.4871, 0.9182],
        [1.1713, 0.9795, 1.2223, 1.6504],
        [2.0215, 1.3112, 1.0539, 1.0291]])

第三种：

```python
'''
Args:
    mean (Tensor): the tensor of per-element means
    std (float, optional): the standard deviation for all distributions
'''
normal(mean, std=1.0, *, out=None) -> Tensor
```
这种情况和第二种情况，恰恰相反了，std变成了每个元素共享的。

In [40]:
torch.normal(mean, 0.5)

tensor([[ 1.1298,  0.1154, -1.1287,  1.5257],
        [-0.2515,  1.6378, -1.0825,  0.2520],
        [ 0.2799,  0.5922, -0.3907, -1.0569]])

第四种：

```python
'''
Args:
    mean (float): the mean for all distributions
    std (float): the standard deviation for all distributions
    size (int...): a sequence of integers defining the shape of the output tensor.
'''
normal(mean, std, size, *, out=None) -> Tensor
```
这种情况下，所有的元素都共享mean和std，最终Tensor的形状是由`size`来决定的

In [41]:
torch.normal(0, 1, (3, 4))

tensor([[-0.3697, -1.0011,  0.2011, -0.3058],
        [-0.0142,  1.7952,  0.3282,  0.6413],
        [-1.2560, -0.4896,  0.7281, -0.1820]])

### torch.rand、torch.randn

`rand`直接生成指定形状的Tensor，其中每个元素都是由`[0,1)`均匀分布来随机产生。

`randn`直接生成指定形状的Tensor，其中每个元素都是由标准正态分布来随机产生。

In [42]:
torch.rand(3, 4)  # 或者 torch.randn((3,4))

tensor([[0.3890, 0.3621, 0.4936, 0.9490],
        [0.4722, 0.2002, 0.4395, 0.1839],
        [0.3801, 0.3354, 0.4724, 0.1334]])

In [43]:
torch.randn(3, 4)  # 或者 torch.randn((3,4))

tensor([[ 0.6906, -0.3491,  0.7373, -0.7658],
        [-0.2358, -1.9815, -0.3103, -0.4552],
        [-0.4182,  0.3479, -2.6860,  0.4807]])

### torch.randint

产生一个由`[low,high)`区间均匀分布随机数填充的LongTensor

```python
randint(low=0,high,size,...)
```

In [44]:
torch.randint(1, 10, (3, 4))

tensor([[6, 9, 6, 2],
        [6, 8, 6, 6],
        [7, 9, 6, 4]])

### torch.randperm

生成一个随机全排列的一维的LongTensor

In [45]:
torch.randperm(12)

tensor([10,  2,  7,  1, 11,  9,  4,  5,  0,  6,  3,  8])

## 使用`xx_like`系列创建相同形态的Tensor

除了shape保持一致外，`dtype`、`layout`、`device`等，若无特别指定，则也与源Tensor保持一致。

```python
torch.zeros_like(input, ..) # 返回与input相同size的零矩阵

torch.ones_like(input, ..) #返回与input相同size的单位矩阵

torch.full_like(input, fill_value, …) #返回与input相同size，单位值为fill_value的矩阵

torch.empty_like(input, …) # 返回与input相同size,并被未初始化的数值填充的tensor

torch.rand_like(input, dtype=None, …) #返回与input相同size的tensor, 填充均匀分布的随机数值

torch.randint_like(input, low=0, high, dtype=None, …) #返回与input相同size的tensor, 填充[low, high)均匀分布的随机数值

torch.randn_like(input, dtype=None, …) #返回与input相同size的tensor, 填充标准正态分布的随机数值

```

In [46]:
src = torch.randn(4, 5)

In [47]:
torch.zeros_like(src)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [48]:
torch.ones_like(src, dtype=torch.int)

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.int32)

In [49]:
torch.empty_like(src, device="cuda:0")

tensor([[1.0000e+00, 2.0000e+00, 3.0000e+00, 4.0000e+00, 5.0000e+00],
        [0.0000e+00, 4.2039e-45, 0.0000e+00, 5.6052e-45, 0.0000e+00],
        [7.0065e-45, 0.0000e+00, 8.4078e-45, 0.0000e+00, 9.8091e-45],
        [0.0000e+00, 1.1210e-44, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       device='cuda:0')

In [50]:
# 这里即使full_value是int类型，但生成的Tensor，依然是用的src的dtype
torch.full_like(src, 42)

tensor([[42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.]])

In [51]:
torch.rand_like(src)

tensor([[0.5314, 0.8197, 0.8071, 0.1591, 0.5537],
        [0.0079, 0.6328, 0.3513, 0.0229, 0.1560],
        [0.3215, 0.3350, 0.4526, 0.0278, 0.4012],
        [0.5541, 0.3188, 0.4859, 0.3072, 0.9894]])

In [52]:
torch.randn_like(src)

tensor([[ 1.1692, -0.8948, -0.4549,  0.9164,  2.5042],
        [-0.5313,  0.0461,  0.1912, -0.9092, -0.2638],
        [-0.2223,  1.7269,  1.8062, -0.3885,  0.8033],
        [-1.1798, -0.3489,  0.1121,  0.5281,  0.6492]])

In [53]:
torch.randint_like(src, 1, 10)

tensor([[9., 9., 1., 1., 5.],
        [9., 5., 8., 7., 3.],
        [2., 1., 1., 5., 6.],
        [2., 3., 2., 2., 9.]])

# Tensor的操作

Pytorch中的Tensor大约支持100种以上的操作，其中包括了数学运算、线性代数、矩阵操作（转置、索引、切片等），这些操作都可以跑在CPU或GPU上，这也是Pytorch Tensor的强大之处。

![](./images/tensor_operatrions.png)

我们可以通过这个[页面](https://pytorch.org/docs/stable/torch.html)，来对Tensor支持的所有操作做个大概的了解。

## 索引访值

### 基础索引

我们可以像访问Numpy.ndarray一样，对torch.Tensor进行各种下标索引与范围切片。

In [54]:
t = torch.arange(12).reshape(3, 4)
print(f"t: {t}")
print(f"取t的第2行的所有元素: {t[1]}")
print(f"取t的最后一列的所有元素: {t[:, -1]}")
print(f"取t的第2列到最后一列的所有元素: {t[:, 2:]}")
print(f"取t的位置(2,3)上的元素: {t[2, 3]}")

t: tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
取t的第2行的所有元素: tensor([4, 5, 6, 7])
取t的最后一列的所有元素: tensor([ 3,  7, 11])
取t的第2列到最后一列的所有元素: tensor([[ 2,  3],
        [ 6,  7],
        [10, 11]])
取t的位置(2,3)上的元素: 11


**单一元素的Tensor**

当我们通过索引访问Tensor的单一元素时，得到的实际是一个`Tensor`类型的对象，它并不是python中的内置数据类型，我们可以通过Tensor的`item()`方法来获取python对象的标量。

In [55]:
type(t[2, 3])

torch.Tensor

In [56]:
t[2, 3].shape

torch.Size([])

In [57]:
type(t[2, 3].item())

int

注意对如果某个维度上我们只取一行/列数据，那么有两种方式，这两种方式得到的结果的 Shape 会不一样

In [58]:
t1 = t[:, 2:3]
t2 = t[:, 2]
print(t1)  # 还是一个二维的Tensor
print(t2)  # 一维的 Tensor

tensor([[ 2],
        [ 6],
        [10]])
tensor([ 2,  6, 10])


### 高级索引

Tensor 的高级索引，支持我们直接用一个 Long 型的 Tensor 作为索引来取原 Tensor 中的元素。

In [59]:
t = torch.randn(8, 10)
# indices 的所有元素都代表 t 的 dim=0 的下标
indices = torch.randint(0, 8, (3, 2))
t[indices].shape

torch.Size([3, 2, 10])

### torch.gather

torch.gather 往往用于我们希望依次在输入 Tensor 的某个维度上取出其中一些索引的值。比如下面的 topK 的结果中，我们希望根据返回的 indices 取得对应的元素，也就是 values，这里可以用 gather

In [60]:
t = torch.randn(3, 5)
values, indices = torch.topk(t, k=2, dim=1)
print(t)
print(values)
print(indices)

tensor([[ 0.5896,  0.2317,  0.5549,  0.5040,  1.3366],
        [ 0.3820,  0.5324,  0.8154, -1.6021,  0.8312],
        [-0.5906, -0.1350,  1.3965,  1.3246,  1.2807]])
tensor([[1.3366, 0.5896],
        [0.8312, 0.8154],
        [1.3965, 1.3246]])
tensor([[4, 0],
        [4, 2],
        [2, 3]])


In [61]:
values == torch.gather(t, dim=1, index=indices)

tensor([[True, True],
        [True, True],
        [True, True]])

## 组合与分片

### torch.cat

```
cat(tensors, dim=0) -> Tensor
```

`torch.cat`将给定义的tensor的序列(tensors)，按给定义的维度上合并起来，这就要求，这些tensor，除了合并的维度，其他的维度必须一致。

In [62]:
t1 = torch.randn(2, 3)
t2 = torch.randn(3, 3)
torch.cat([t1, t2], dim=0)

tensor([[-0.8458,  0.5044, -0.2978],
        [ 1.0015,  0.3516, -0.6523],
        [-0.8931, -1.1678,  0.1994],
        [ 0.4824,  0.0945, -0.2003],
        [-1.1476, -0.6038,  0.5421]])

### torch.stack

`torch.stack`和`torch.cat`接口用法一致，但它并不是在原有的维度上拼接，而是直接扩展一个新的维度。

这就要求，序列中的tensor在维度上必须一致。

In [63]:
t1 = torch.randn(2, 3)
t2 = torch.randn(2, 3)
torch.stack([t1, t2], dim=0)

tensor([[[ 0.6103,  0.7689, -1.8726],
         [-0.8682,  0.2326,  0.8563]],

        [[ 1.8399, -1.2037,  0.5311],
         [ 0.3968,  1.1138,  1.1531]]])

### torch.split
```python
split(tensor, split_size_or_sections, dim=0)
```
`split`将tensor按指定的维度，分拆为多个Tensor的元组，拆分的块chunk的大小是splite_size指定的。可能出现不能整分的情况，这时候最后一块大小一般小于splite_size

split出来的Tensor是原tensor的一个view

In [64]:
a = torch.arange(10).view(5, 2)
a

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [65]:
torch.split(a, 2)

(tensor([[0, 1],
         [2, 3]]),
 tensor([[4, 5],
         [6, 7]]),
 tensor([[8, 9]]))

`split_size_or_sections`也可能是一个list(int)，这时候，它的每个元素，代表每个chunk的大小

In [66]:
a1, a2, a3 = torch.split(a, (1, 3, 1))
a1, a2, a3

(tensor([[0, 1]]),
 tensor([[2, 3],
         [4, 5],
         [6, 7]]),
 tensor([[8, 9]]))

In [67]:
# 切分出来的tensor和原tensor是共享存储的
a1[0, 0] = 42
a

tensor([[42,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9]])

### torch.chunk

```python
chunk(input, chunks, dim=0) -> List of Tensors
```
`chunk`和`split`功能类似，不同在于，chunk的第二的参数，直接指定的是chunk的数量，最后一个chunk的数量可能会少一些。也有可能`axis[dim]<chunks`，那么就直接切分为`axis[dim]`个。

切分出来的这些Tensor和原Tensor都是共享底层存储的，也就是说每个chunk都是原Tensor的一个view。

In [68]:
print(a.shape)
len(a.chunk(3, dim=1))

torch.Size([5, 2])


2

## 变换操作

### torch.reshape

```python
reshape(input, shape) -> Tensor
```
`reshape`返回一个和原Tensor具有相同数据，相同数量的Tensor，只是shape不一致。

### torch.view

torch.view vs. torch.reshape

`reshape`可以用在`compact`或`non-compact`的tensor上，而`view`只能用在`compact`的tensor上。`reshape`如果作用于`non-compact`的tensor上，则会产生一个copy

torch.view has existed for a long time. It will return a tensor with the new shape. The returned tensor will share the underling data with the original tensor. See the documentation here.

On the other hand, it seems that torch.reshape has been introduced recently in version 0.4. According to the document, this method will

> Returns a tensor with the same data and number of elements as input, but with the specified shape. When possible, the returned tensor will be a view of input. Otherwise, it will be a copy. Contiguous inputs and inputs with compatible strides can be reshaped without copying, but you should not depend on the copying vs. viewing behavior.

It means that torch.reshape may return a copy or a view of the original tensor. You can not count on that to return a view or a copy. According to the developer:

> if you need a copy use clone() if you need the same storage use view(). The semantics of reshape() are that it may or may not share the storage and you don't know beforehand.

Another difference is that reshape() can operate on both contiguous and non-contiguous tensor while view() can only operate on contiguous tensor. Also see here about the meaning of contiguous.

### torch.transpose

```python
transpose(input, dim0, dim1) -> Tensor
```
转置input的指定的2个维度，返回的Tensor和原来的Tensor共享存储

In [69]:
x = torch.rand(2, 3, 4)
x

tensor([[[0.1958, 0.6966, 0.5887, 0.5991],
         [0.2259, 0.0063, 0.9598, 0.5841],
         [0.0613, 0.7575, 0.9952, 0.8753]],

        [[0.5962, 0.9140, 0.4718, 0.9588],
         [0.5693, 0.2644, 0.6633, 0.6033],
         [0.5709, 0.3676, 0.2916, 0.1691]]])

In [70]:
y = torch.transpose(x, 0, 2)

In [71]:
print(x.stride())
print(y.stride())

(12, 4, 1)
(1, 4, 12)


### torch.permute

In [72]:
torch.permute(x, (2, 0, 1)).shape

torch.Size([4, 2, 3])

### squeeze和unsqueeze

squeeze在指定的维度上添加一维，而unsqueeze则在指定的维度上去掉`size=1`的维度，如果对应维度上的size不等于1，则不做任何操作

In [73]:
x = torch.randn(2, 3)
y = x.unsqueeze(dim=1).unsqueeze(dim=0)
print(f"x shape: {x.shape}, \ny (x.unsqueeze) shape: {y.shape}")
print("unsqueeze shape", y.squeeze(dim=0).shape)

x shape: torch.Size([2, 3]), 
y (x.unsqueeze) shape: torch.Size([1, 2, 1, 3])
unsqueeze shape torch.Size([2, 1, 3])


### contiguous

There are a few operations on Tensors in PyTorch that do not change the contents of a tensor, but change the way the data is organized. These operations include:

`narrow()`, `view()`, `expand()` and `transpose()`

For example: when you call transpose(), PyTorch doesn't generate a new tensor with a new layout, it just modifies meta information in the Tensor object so that the offset and stride describe the desired new shape. In this example, the transposed tensor and original tensor share the same memory:

In [74]:
x = torch.randn(3, 2)
y = torch.transpose(x, 0, 1)
x[0, 0] = 42
print(y[0, 0])

tensor(42.)


This is where the concept of contiguous comes in. In the example above, x is contiguous but y is not because its memory layout is different to that of a tensor of same shape made from scratch. Note that the word "contiguous" is a bit misleading because it's not that the content of the tensor is spread out around disconnected blocks of memory. Here bytes are still allocated in one block of memory but the order of the elements is different!

When you call contiguous(), it actually makes a copy of the tensor such that the order of its elements in memory is the same as if it had been created from scratch with the same data.

Normally you don't need to worry about this. You're generally safe to assume everything will work, and wait until you get a RuntimeError: input is not contiguous where PyTorch expects a contiguous tensor to add a call to contiguous().

## 降维操作

### torch.mean

```python
'''
Args:
  input (Tensor): the input tensor.
  dim (int or tuple of ints): the dimension or dimensions to reduce.
  keepdim (bool): whether the output tensor has :attr:`dim` retained or not.
'''
mean(input, dim, keepdim=False, *, out=None) -> Tensor
```

对input沿着`dim`的维度求均值，这样的话，指定的那个维度就会被压缩掉，如果指定了`keepdim=True`的话，那个维度会保留，值为1

In [75]:
t = torch.randn(5, 6)
t

tensor([[ 0.7664, -0.8458,  0.2225, -1.0404,  0.9716,  0.6827],
        [-0.6373,  1.3663,  0.1353,  1.0554,  0.5993, -1.8393],
        [-0.2648,  0.2339,  0.9827,  1.4707, -0.4960,  0.0141],
        [ 1.3204, -0.9046, -0.0559,  1.2988, -0.3727, -0.7442],
        [-1.5427, -1.3474, -0.8282,  0.7764,  2.6069, -0.3903]])

In [76]:
# 按列的方向(dim=0)将整个Tenoor压缩成为1维的
torch.mean(t, dim=0)

tensor([-0.0716, -0.2995,  0.0913,  0.7122,  0.6618, -0.4554])

In [77]:
torch.mean(t, dim=1, keepdim=True)

tensor([[ 0.1262],
        [ 0.1133],
        [ 0.3234],
        [ 0.0903],
        [-0.1209]])

对于高维Tensor，我们还可以同时对多个维度进行Reduce，求其均值。

In [78]:
t = torch.randn(2, 3, 4)
t

tensor([[[-0.6024,  0.4861, -0.3549,  1.2723],
         [ 0.7566, -0.3582, -1.0746,  2.0221],
         [-0.5265, -1.8632, -1.2744, -0.2883]],

        [[-0.9824,  1.8946, -0.5908, -0.8086],
         [ 0.2823, -0.5457,  0.9007,  1.5686],
         [ 0.2141,  0.1007,  0.3016,  0.2697]]])

In [79]:
# 等价于reduce第0维，得到一个3x4的Tensor后，再reduce第1维，得到(3,)的Vector
torch.mean(t, dim=(0, 2))

tensor([ 0.0392,  0.4440, -0.3833])

In [80]:
t.mean(0).mean(1)

tensor([ 0.0392,  0.4440, -0.3833])

### torch.sum

`torch.sum`是一个和`torch.mean`用法上很像的操作，只是`sum`的reduce op变成了求和，而不是求均值。

In [81]:
torch.sum(t, dim=(0, 2))

tensor([ 0.3139,  3.5517, -3.0662])

### torch.argmax

In [82]:
x = torch.rand((2, 3))
print("x:", x)
print("Argmax:", x.argmax(dim=1))

x: tensor([[0.5064, 0.0923, 0.3140],
        [0.4148, 0.5013, 0.0254]])
Argmax: tensor([0, 1])


### torch.maxmimu

相同 Shape 的 Tensor 和 Tensor 按元素逐个比大小，保留最大的

In [83]:
def relu(x):
    return torch.maximum(x, torch.tensor(0))


x = torch.randn((2, 3))
print(f"x:\n\t{x} \nrelu(x):\n\t{relu(x)}")

x:
	tensor([[ 0.2393, -0.8319,  0.1255],
        [ 1.7573, -1.0650,  0.1229]]) 
relu(x):
	tensor([[0.2393, 0.0000, 0.1255],
        [1.7573, 0.0000, 0.1229]])


## 排序

### torch.sort

```python
sort(input, dim=-1, descending=False, *, out=None) -> (Tensor, LongTensor)
```
`sort`对input按给定义的dim进行升序排列，返回排列后的Tensor的同时，也返回一个对应的下标的重排后的Tensor

dim的默认值是Tensor的最后一维

In [84]:
a = torch.rand(2, 3)
print(a)
values, indices = torch.sort(a, dim=1, descending=True)
print(values)
print(indices)

tensor([[0.6248, 0.7040, 0.7312],
        [0.9658, 0.5388, 0.5723]])
tensor([[0.7312, 0.7040, 0.6248],
        [0.9658, 0.5723, 0.5388]])
tensor([[2, 1, 0],
        [0, 2, 1]])


### torch.topk

```python
topk(input, k, dim=None, largest=True, sorted=True, *, out=None) -> (Tensor, LongTensor)
```
`topk`返回input中指定维度上，最大的k个元素，以及对应的索引。

In [85]:
a = torch.randn(5)
a

tensor([-1.1304, -1.0381, -1.1502,  1.2661,  0.4508])

In [86]:
values, indices = torch.topk(a, 3)
print(values)
print(indices)

tensor([ 1.2661,  0.4508, -1.0381])
tensor([3, 4, 1])


### torch.kthvalue

```python
kthvalue(input, k, dim=None, keepdim=False, *, out=None) -> (Tensor, LongTensor)
```
`kthvalue`计算输出Tensor的指定维度上第`k`小的元素以及下标。如果dim没有指定，则默认为Tensor的最后一维。

In [87]:
a = torch.randn(4, 3)
a

tensor([[ 0.3197,  0.4240,  0.3617],
        [-2.1741,  2.1711,  0.4256],
        [ 0.5445, -0.3068, -1.1504],
        [ 0.5853, -0.8309,  1.4965]])

In [88]:
torch.kthvalue(a, 2, dim=0)

torch.return_types.kthvalue(
values=tensor([ 0.3197, -0.3068,  0.3617]),
indices=tensor([0, 2, 0]))

## 原地操作(in-place)

pytorch的Tensor支持了很多原地操作，它们的特点就是在方法末尾以`_`结束

In [89]:
t1 = torch.ones(2, 3)
print(f"t1 = {t1}")
t1.add_(2)
print(f"after plus 2: t1 = {t1}")

t1 = tensor([[1., 1., 1.],
        [1., 1., 1.]])
after plus 2: t1 = tensor([[3., 3., 3.],
        [3., 3., 3.]])


## 转换为其他数据类型

我们可以调用`numpy`接口,返回一个numpy.ndarray的对象，可以调用`tolist`接口，返回一个list的对象

In [90]:
t = torch.tensor([1, 2, 3, 4, 5, 6])

In [91]:
# 返回的ndarray还是和t是共享存储的
t.numpy()

array([1, 2, 3, 4, 5, 6])

In [92]:
t.reshape(2, 3).tolist()

[[1, 2, 3], [4, 5, 6]]

## repeat和repeat_interleave

In [93]:
a = torch.arange(6).reshape((2, 3))
a

tensor([[0, 1, 2],
        [3, 4, 5]])

`repeat(d0, d1, d2)` 将对应的维度复制多份，如果之前没有对应的维度，则可以当作原来维度为1，处理。

In [94]:
a.repeat((2, 1, 2))

tensor([[[0, 1, 2, 0, 1, 2],
         [3, 4, 5, 3, 4, 5]],

        [[0, 1, 2, 0, 1, 2],
         [3, 4, 5, 3, 4, 5]]])

`repeat_interleave(n, dim)` 在对应的维度上进行复制，但复制的方式不是`[a b c a b c ]`这种，而是`[a a b b c c]`

In [95]:
a.repeat_interleave(2, dim=0)

tensor([[0, 1, 2],
        [0, 1, 2],
        [3, 4, 5],
        [3, 4, 5]])

# 爱因斯坦标识

In [12]:
from einops import rearrange, reduce, repeat

## 实现 Transpose 和 Permute 的功能

In [16]:
x = torch.randn(2, 3, 8, 8)

# Transose
torch.allclose(rearrange(x, "b c h w->b h w c"), x.permute((0, 2, 3, 1)))

True

## 一步实现 Transpose + Reshape

In [19]:
# 一步完成 Transpose + Reshape
torch.allclose(
    rearrange(x, "b c h w -> (b h w) c"), x.permute(0, 2, 3, 1).reshape(-1, x.size(1))
)

True

## 实现维度拆分

In [24]:
x = torch.randn(2, 3, 64)

torch.allclose(rearrange(x, "b c (h w) -> b c h w", h=8), x.reshape(2, 3, 8, 8))

True

## 实现 Image2Patch 的功能

将 二维图像转换为 $B\times N \times D$ 的序列 Patches 的形式。

In [29]:
image = torch.randn(2, 3, 256, 256)

patches = rearrange(image, "b c (h1 ph) (w1 pw) -> b (h1 w1) (ph pw c)", ph=8, pw=8)
print(patches.shape)

torch.Size([2, 1024, 192])


## Reduce 操作

In [32]:
x = torch.randn(8, 10)

# mean
x_mean = reduce(x, "b d -> b", reduction="mean")
# sum
x_sum = reduce(x, "b d -> 1 d", reduction="sum")

x = torch.randn(2, 3, 4)
x_max = reduce(x, "b n d -> d", reduction="max")

## 扩维与复制

In [33]:
x = torch.randn(5, 5)
rearrange(x, "i j -> 1 i j").shape

torch.Size([1, 5, 5])

In [35]:
x = torch.randn(1, 5, 5)
repeat(x, "1 i j -> 3 i (5 j)").shape

torch.Size([3, 5, 25])